In [1]:
# !pip install -r requirements.txt
# langgraph
# langchain-community
# langchain-core
# langchain-openai
# langchain
# langgraph-sdk
# langgraph-checkpoint-sqlite
# langsmith
# langchainhub==0.1.15
# langchain-openai==0.1.3
# langchain==0.1.16
# langchain-core==0.1.42
# pygraphviz==1.12
# llama-index
# fastapi
# uvicorn

!pip install -qU langgraph langchain-community langchain-core langchain-openai langchain langgraph-sdk langgraph-checkpoint-sqlite langsmith langchainhub==0.1.15 langchain-openai==0.1.3 langchain==0.1.16 langchain-core==0.1.42 pygraphviz==1.12 llama-index fastapi uvicorn openai

ERROR: Cannot install langchain-core, langchain-core==0.1.42, langchain-openai==0.1.3, langchain==0.1.16, langgraph==0.0.39, langgraph==0.0.40, langgraph==0.0.41, langgraph==0.0.42, langgraph==0.0.43, langgraph==0.0.44, langgraph==0.0.45, langgraph==0.0.46, langgraph==0.0.47, langgraph==0.0.48, langgraph==0.0.49, langgraph==0.0.50, langgraph==0.0.51, langgraph==0.0.52, langgraph==0.0.53, langgraph==0.0.54, langgraph==0.0.55, langgraph==0.0.56, langgraph==0.0.57, langgraph==0.0.58, langgraph==0.0.59, langgraph==0.0.60, langgraph==0.0.61, langgraph==0.0.62, langgraph==0.0.63, langgraph==0.0.64, langgraph==0.0.65, langgraph==0.0.66, langgraph==0.0.67, langgraph==0.0.68, langgraph==0.0.69, langgraph==0.1.1, langgraph==0.1.10, langgraph==0.1.11, langgraph==0.1.12, langgraph==0.1.13, langgraph==0.1.14, langgraph==0.1.15, langgraph==0.1.16, langgraph==0.1.17, langgraph==0.1.19, langgraph==0.1.2, langgraph==0.1.3, langgraph==0.1.4, langgraph==0.1.5, langgraph==0.1.6, langgraph==0.1.7, langgrap

# SETUP Enviornment

In [2]:
import os.path
import logging
import sys

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
# Viewing Queries and Events Using Logging
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# loading env variables 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = "OPEN AI KEY TO BE PUT HERE"
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# CSV Generation

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from pydantic import BaseModel, Field
from langgraph.graph.message import AnyMessage, add_messages
from typing import Literal, Annotated
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, List
from openai import OpenAI
import os 
from langchain_openai import ChatOpenAI
import yaml

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Define the state
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

openai_api_key = os.getenv("OPENAI_API_KEY")
BOT_USED = ""
# Load prompts from YAML
def load_prompts(file_path='prompts.yaml'):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

prompts = load_prompts()

# Initialize OpenAI model
model = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Define prompts for different agents
planner_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a planner agent that decides which specialized agent to call based on the user's input. Respond with one of 'suicide_prevention', 'conversational', 'anger_management', 'motivational', or 'mindfulness' based on the user's emotion."),
    ("human", "{input}"),
])

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['empathetic_agent_prompt']),
    ("human", "{input}"),
])

suicide_prevention_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['suicide_prevention_agent_prompt']),
    ("human", "{input}"),
])


anger_management_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['anger_prevention_agent_prompt']),
    ("human", "{input}"),
])

motivational_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['motivational_agent_prompt']),
    ("human", "{input}"),
])

mindfulness_prompt = ChatPromptTemplate.from_messages([
    ("system", prompts['mindfulness_agent_prompt']),
    ("human", "{input}"),
])

# Define router
def route_query(state: State):
    class RouteQuery(BaseModel):
        """Route a user query to the most relevant node."""
        route: Literal["conversational", "suicide_prevention", "anger_management", "motivational", "mindfulness"] = Field(
            ...,
            description="Choose the appropriate agent based on the user's emotions."
        )
    
    structured_llm_router = model.with_structured_output(RouteQuery)
    question_router = planner_prompt | structured_llm_router
    last_message = state["messages"][-1]
    resp = question_router.invoke({"input": last_message})
    return resp.route

# Define agent functions
def run_conversational_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running conversational agent")
    convo_model = conversational_prompt | model
    response = convo_model.invoke(state["messages"])
    BOT_USED = "conversational_agent"  # Assign to global variable
    return {"messages": response}

def run_suicide_prevention_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running suicide prevention agent")
    concern_model = suicide_prevention_prompt | model
    response = concern_model.invoke(state["messages"])
    BOT_USED = "suicide_agent"  # Assign to global variable
    return {"messages": response}

def run_anger_management_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running anger management agent")
    anger_model = anger_management_prompt | model
    response = anger_model.invoke(state["messages"])
    BOT_USED = "anger_agent"  # Assign to global variable
    return {"messages": response}

def run_motivational_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running motivational agent")
    motivation_model = motivational_prompt | model
    response = motivation_model.invoke(state["messages"])
    BOT_USED = "motivational_agent"  # Assign to global variable
    return {"messages": response}

def run_mindfulness_agent(state: State):
    global BOT_USED  # Declare BOT_USED as global
    print("Running mindfulness agent")
    mindfulness_model = mindfulness_prompt | model
    response = mindfulness_model.invoke(state["messages"])
    BOT_USED = "mindfulness_agent"  # Assign to global variable
    return {"messages": response}


# Create the graph
workflow = StateGraph(State)

# Add nodes for each agent
workflow.add_node("conversational", run_conversational_agent)
workflow.add_node("suicide_prevention", run_suicide_prevention_agent)
workflow.add_node("anger_management", run_anger_management_agent)
workflow.add_node("motivational", run_motivational_agent)
workflow.add_node("mindfulness", run_mindfulness_agent)

# Add edges
workflow.add_conditional_edges(
    START,
    route_query,
    {
        "conversational": "conversational",
        "suicide_prevention": "suicide_prevention",
        "anger_management": "anger_management",
        "motivational": "motivational",
        "mindfulness": "mindfulness"
    }
)
workflow.add_edge("conversational", END)
workflow.add_edge("suicide_prevention", END)
workflow.add_edge("anger_management", END)
workflow.add_edge("motivational", END)
workflow.add_edge("mindfulness", END)

# Compile the graph
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

# Function to run a conversation turn
def chat(message: str, config: dict):
    result = graph.invoke({"messages": [HumanMessage(content=message)]}, config=config)
    return result

if __name__ == "__main__":
    config = {"configurable": {"thread_id": "1"}}
        
    # Open a CSV file for writing
    with open('chat_responses.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["User Input", "Triggered Agent", "Response"])  # Write the header

        # i = 0
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Bot: Goodbye!")
                break
                
            response = chat(user_input, config)
            # response = response["messages"].content
            response = response['messages'][-1].content
            print(f"Sukoon: {response}")

            # Log the interaction to the CSV
            writer.writerow([user_input, BOT_USED, response])  # Write the data
            print(BOT_USED,"**********")
            BOT_USED = ""

    

In [8]:
response

AIMessage(content="I understand you're feeling angry right now, and it's okay to feel that way. Can you tell me what triggered your anger today? Would you like to try some deep breathing exercises to calm down? What small change could help you feel more in control of your emotions right now?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 1454, 'total_tokens': 1509, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1024}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-97f50719-c7e7-4b37-b2ea-3a484a066fc0-0', usage_metadata={'input_tokens': 1454, 'output_tokens': 55, 'total_tokens': 1509, 'input_token_details': {'cache_read': 1024}, 'output_token_details': {'reasoning': 0}})